In [1]:
from PIL import Image, ImageChops, ImageColor
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math

In [2]:
class ContourModel:
    def __init__(self, x = 0, y = 0, w = 0, h = 0):
        self.x = x
        self.y = y
        self.w = w
        self.h = h 

Поиск искр от сварки на кадре

In [4]:
def detect_for_color(frame, lower, upper):    
    hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)


    mask = cv.inRange(hsv_frame, lower, upper)
    target = cv.bitwise_and(hsv_frame, hsv_frame, mask=mask)
    blur = cv.GaussianBlur(mask, (0, 0), sigmaX=10, sigmaY=10)
    
    ret, thresh = cv.threshold(blur, 40, 255, 0)
    
    #cv.imshow('asd', thresh)
    cont, hier = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    
    if len(cont) != 0:
        c = max(cont, key=cv.contourArea)
        cv.drawContours(target, c, -1, 255, 3)
        x, y, w, h = cv.boundingRect(c)
        return ContourModel(x, y, w, h)

    return ContourModel()
    

Контуры движущихся предметов

In [5]:
def detect_movement(frame1, frame2) -> list[ContourModel]:
    bodies = []
    #cap = cv.VideoCapture('right_half.mp4')
    #ret, frame1 = cap.read()
    #ret, frame2 = cap.read()  
    #while cap.isOpened():
    
    diff = cv.absdiff(frame1, frame2)
    diff_gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(diff_gray, (0, 0), sigmaX=3, sigmaY=3)

    _, thresh = cv.threshold(blur, 25, 255, 0)
    dilated = cv.dilate(thresh, None, iterations=20)   
    #cv.imshow('asd', diff)
    conts, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)  

    for cont in conts:
        x, y, w, h = cv.boundingRect(cont)
        area = cv.contourArea(cont)
        if area < 40000 and area > 4000:
            bodies.append(ContourModel(x, y, w, h))
            #cv.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
            #cv.putText(frame1, "Ped{}".format('Movement'), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
               
    return bodies

#cap.release()
#cv.destroyAllWindows()

In [7]:
cap = cv.VideoCapture('./example/learn_video.mp4')
while True:
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()

    y = frame1.shape[0]
    x = frame1.shape[1]

    human = detect_movement(frame1, frame2)
    sides = [0, 0, frame1.shape[0], frame1.shape[1]]
    for cont in human:
        containers = [cont.x, cont.y, cont.x + cont.w, cont.y + cont.h]

        for i, side in enumerate(sides):
            if side and i in [0, 1] and containers[i] * 0.9 >= side:
                containers[i] = int(0.9 * containers[i])
            elif containers[i] * 1.1 <= side:
                containers[i] = int(1.1 * containers[i])

        cropped = frame1[containers[1] : containers[-1],
                         containers[0] : containers[2]]
        
        svar_spark_lower = np.array([85, 125, 180])
        svar_spark_upper = np.array([140, 225, 255])
        svar_spark = detect_for_color(cropped, svar_spark_lower, svar_spark_upper)

        bolg_spark_lower = np.array([0,0,255])
        bolg_spark_upper = np.array([23,90,255])
        bolg_spark = detect_for_color(cropped, bolg_spark_lower, bolg_spark_upper)
        # cv.rectangle(frame1, (cont.x, cont.y),
        #              (cont.x + cont.w,
        #               cont.y + cont.h),
        #              (0, 0, 255), 2)
        #print(spark.x, spark.y, spark.x + spark.w, spark.y + spark.h)
        cv.rectangle(frame1, (containers[0] + svar_spark.x, containers[1] + svar_spark.y), 
                             (containers[0] + svar_spark.x + svar_spark.w, containers[1] + svar_spark.y + svar_spark.h),
                             (0, 255, 0), 2)
        cv.rectangle(frame1, (containers[0] + bolg_spark.x, containers[1] + bolg_spark.y), 
                     (containers[0] + bolg_spark.x + bolg_spark.w,
                      containers[1] + bolg_spark.y + bolg_spark.h),
                     (0, 0, 255), 2)


    cv.imshow('asd', frame1)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

: 

In [7]:
img = cv.imread('./test.png')
detect_for_color(img)
    
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
import cv2
import numpy as np


def nothing(x):
    pass


# Load image
image = cv2.imread('./svarka.png')

# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
# Hue is from 0-179 for Opencv
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

while (1):
    # Get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Convert to HSV format and color threshold
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if ((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax)):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (
            hMin, sMin, vMin, hMax, sMax, vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display result image
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

(hMin = 0 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 51), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 52), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 53), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 54), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 55), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 57), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 63), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 71), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 80), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 89), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 96), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 101), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 0 , sMin = 0, vMin = 104), (hMax = 179 , sMax = 255, vMa